# Tutorial: PA2 – Unsupervised Learning

## Part 1: Image Segmentation with Clustering
Task 1.1: Implementing K-means++ Initialization, K-means, and K-medoids

For kmeanspp_init function:

In [ ]:
# Key steps breakdown:
# 1. Initialize array to hold k centroids
# 2. Pick first centroid randomly
# 3. For remaining k-1 centroids:
#    - Compute squared distances from each point to nearest centroid
#    - Convert distances to probabilities (normalize)
#    - Sample next centroid using these probabilities

# Useful NumPy functions:
# - np.random.randint(n) : random index
# - np.linalg.norm(X - centroid, axis=1) : distances from all points to one centroid
# - np.minimum(arr1, arr2) : element-wise minimum
# - np.random.choice(n, p=probabilities) : weighted random sampling

: 

K-means Algorithm:

In [ ]:
# Algorithm structure:
# 1. Initialize centroids (using init parameter)
# 2. Repeat until convergence:
#    a. Assignment step: assign each point to nearest centroid
#    b. Update step: recalculate centroids as cluster means
# 3. Check convergence (centroids don't change)

# Key vectorization trick for assignment:
# np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
# This creates distance matrix: shape (n_points, k_centroids)
# Broadcasting: X[:, np.newaxis] creates shape (n, 1, d)
#              centroids has shape (k, d)
#              subtraction broadcasts to (n, k, d)
#              norm along axis=2 gives (n, k)

# Useful functions:
# - np.argmin(distances, axis=1) : find nearest centroid for each point
# - np.allclose(arr1, arr2) : check if arrays are approximately equal
# - np.any(labels == i) : check if cluster i is non-empty

K-medoids Algorithm:


In [ ]:
# Algorithm structure:
# 1. Initialize k medoids randomly from dataset
# 2. Repeat until convergence:
#    a. Assignment: assign points to nearest medoid
#    b. Update: for each cluster, find point that minimizes total distance
#       to all other points in cluster (new medoid)
# 3. Check convergence (medoid indices don't change)

# Finding new medoid for cluster i:
# - Get all points in cluster i
# - Compute pairwise distance matrix within cluster
# - Sum distances for each point (to all others in cluster)
# - Point with minimum sum becomes new medoid

# Useful functions:
# - np.where(labels == i)[0] : get indices of points in cluster i
# - np.linalg.norm(points[:, np.newaxis] - points, axis=2) : pairwise distances
# - np.sum(dist_matrix, axis=1) : sum distances for each point
# - np.argmin(sums) : find point with minimum total distance

Task 1.2: Calculating WCSS (Within-Cluster Sum of Squares)

In [ ]:
# For calculate_wcss function:
# 1. Initialize wcss = 0
# 2. For each cluster i:
#    a. Get all points assigned to cluster i
#    b. Get centroid/medoid for cluster i
#    c. Compute squared distances: (point - center)^2
#    d. Sum these distances and add to wcss
# 3. Return total wcss

# Useful functions:
# - np.unique(labels) : get number of clusters
# - X[labels == i] : filter points in cluster i
# - np.sum(np.square(differences), axis=1) : squared Euclidean distance
# - Handle empty clusters: check len(cluster_points) > 0

## Part 2: Anomaly Detection in Credit Card Transactions
Task 2.1: Exploratory Data Analysis (EDA)

In [ ]:
# Suggested analyses:
# 1. Class distribution
#    - Count fraud vs normal transactions
#    - Visualize imbalance (bar plot)

# 2. Feature distributions
#    - Histograms for V1-V28 features
#    - Compare fraud vs normal (overlaid distributions)

# 3. Feature correlations
#    - Correlation matrix heatmap
#    - Identify highly correlated features

# 4. Amount analysis
#    - Box plots: fraud vs normal transaction amounts
#    - Statistical summary

Task 2.2: Implementing DBSCAN
-  For region_query function:

In [1]:
# Purpose: Find all points within ε distance of a given point
# Input: point_idx (index of query point)
# Output: array of indices of neighbors

# Implementation:
# 1. Compute distances from query point to all points
#    dists = np.linalg.norm(X - X[point_idx], axis=1)
# 2. Return indices where distance <= eps
#    return np.where(dists <= eps)[0]

-  For expand_cluster function:

In [ ]:
# Purpose: Grow cluster by adding density-reachable points
# Uses: nonlocal cluster_id (to modify outer scope variable)

# Algorithm:
# 1. Assign current point to cluster_id
# 2. Iterate through neighbors (use while loop, not for):
#    - If neighbor is noise (-1), assign to cluster
#    - If neighbor unprocessed:
#      - Query its neighbors
#      - If it's a core point (enough neighbors):
#        - Add its neighbors to expansion list
# 3. Continue until no more points to add

# Key insight: Use while loop because neighbors list grows during iteration

-  Main DBSCAN loop:

In [ ]:
# For each unvisited point:
# 1. Find its neighbors
# 2. If not enough neighbors: mark as noise (-1)
# 3. Otherwise: expand_cluster and increment cluster_id

# Note: Points initially marked as noise can later be 
# assigned to clusters if they're density-reachable from core points

-  Example

In [ ]:
# Simple 2D example with eps=1.5, min_samples=2
# Points: A(0,0), B(1,0), C(5,5), D(6,5), E(10,10)
# 
# Process A: neighbors={A,B} (2 points) → core point → cluster 0
# Process B: already in cluster 0
# Process C: neighbors={C,D} (2 points) → core point → cluster 1  
# Process D: already in cluster 1
# Process E: neighbors={E} (1 point) → noise (-1)

## Part 3: Skin Detection via YCbCr + GMM
Task 3.1: Implementing GMM with EM Algorithm
-  For gaussian_pdf function:

In [ ]:
# Implements multivariate Gaussian PDF formula
# Inputs: X (n, d), mean (d,), cov (d, d)
# Output: array of probabilities (n,)

# Steps:
# 1. Compute differences: diff = X - mean
# 2. Compute inverse covariance: inv_cov = np.linalg.inv(cov)
# 3. Compute exponent term: (diff @ inv_cov * diff).sum(axis=1)
#    - This efficiently computes (x-μ)ᵀΣ⁻¹(x-μ) for all points
# 4. Compute determinant: det = np.linalg.det(cov)
# 5. Combine into formula

# Useful trick: diff @ inv_cov * diff is efficient element-wise product
# after matrix multiplication, equivalent to:
# np.sum(diff @ inv_cov * diff, axis=1)

-  For gmm_em function:

In [ ]:
# Initialization:
# - weights: uniform (1/k for each component)
# - means: randomly sample k points from data
# - covs: list of k covariance matrices (initialize with data covariance)

# EM Loop:
# E-step:
#   1. Compute likelihoods: lik[i,k] = gaussian_pdf for component k
#   2. Weight likelihoods: weighted_lik = lik * weights
#   3. Compute responsibilities: resp = weighted_lik / sum(weighted_lik)
#      - resp[i,k] = probability point i belongs to component k

# M-step:
#   1. Compute effective number of points: nk = sum(resp, axis=0)
#   2. Update weights: weights = nk / n
#   3. Update means: means[k] = weighted average of points
#      means[k] = (resp[:,k] @ X) / nk[k]
#   4. Update covariances:
#      - Compute differences from mean
#      - Weight by responsibilities
#      - Add regularization: + np.eye(d) * 1e-6

# Convergence:
# - Compute log-likelihood: sum(log(weighted_lik.sum(axis=1)))
# - Stop if change < tol or max_iter reached

# Why regularization? Prevents singular covariance matrices
# (matrices that can't be inverted)

---

## SAMPLE APPROACHES TO ANALYTICAL QUESTIONS

These aren't trick questions—they're asking you to think about what you've observed, to experiment with parameters, and to explain why certain algorithms work better in certain contexts. The key here is to ground your answers in evidence: run experiments, create visualizations, reference your code outputs, or explain the underlying mathematics.

**Note:** The following approaches are just ideas, you are **not required** to actually follow these samples exactly, you have liberty to design your own experiments as well. Also as mentioned in the main notebook, you can also refer back to the Core Tasks themselves if you feel they are sufficient to supplement your analysis.

-  In this tutorial, we shall cover just a couple of questions to give you an idea of what can be done for these parts

Question 1a: Effect of varying k in K-means++

In [ ]:
# Run K-means++ with k = 3, 5, 7, 10
# For each k:
#   - Compute WCSS (decreases with k)
#   - Compute MSE between original and segmented image
#   - Visual inspection: count distinct regions

def compute_mse(original, segmented):
    # Compute mean squared error between original and segmented images
    pass

# Create comparison plot:
# X-axis: k values
# Y-axis: WCSS and MSE
# Discuss: elbow point, diminishing returns

Question 1d: Algorithm comparison

In [ ]:
# Create comparison table from metrics
# Analyze trade-offs:
# - K-means++: fast, sensitive to initialization
# - K-medoids: robust to outliers, slower
# - Agglomerative: no need for k, hierarchical structure

# Discuss why scores differ:
# - Initialization quality
# - Handling of outliers
# - Cluster shape assumptions

---

Question 2a: Effect of k on fraud detection


In [ ]:
# Run K-means++ with k = 2, 3, 4, 5
# For each k:
#   - Identify anomaly cluster (smallest)
#   - Compute precision, recall, F1
#   - Plot metrics vs k

# Expected observation:
# - Higher k may split fraud cluster
# - Recall dilution as fraud points distributed across clusters

Question 2b: DBSCAN hyperparameter tuning


In [ ]:
# For each combination:
#   - Run DBSCAN
#   - Compute metrics
#   - Create heatmap of recall vs (eps, min_samples)

# Discuss trade-off:
# - Tighter settings: fewer false positives, may miss fraud
# - Looser settings: more noise detected as fraud

---

Question 3a: GMM components variation

In [ ]:
# Fit GMMs with k = 2, 3, 4, 5 components (just as an example)
# For each k:
#   - Compute likelihood ratio
#   - Generate skin mask
#   - Compute true/false positive rates (if ground truth available)
#   - Count bounding boxes

# Discuss:
# - More components: better fit, but risk overfitting
# - Optimal k balances flexibility and generalization

Question 3b: EM convergence parameters


In [ ]:
# Test combinations:
tolerances = [1e-3, 1e-4, 1e-5, 1e-6]
max_iters = [50, 100, 200]

# For each combination:
#   - Measure fitting time
#   - Compute final log-likelihood
#   - Evaluate mask quality (visual inspection or metrics)

Question 3e: Likelihood ratio threshold

In [ ]:
# Experiment with thresholds: [0.5, 1.0, 2.0, 5.0, 10.0]
# For each:
#   - Generate skin mask
#   - Count and visualize bounding boxes
#   - Compute average confidence per box

# Create ROC-like analysis:
# - Lower threshold: high sensitivity, low specificity
# - Higher threshold: low sensitivity, high specificity

# Discuss extraneous boxes:
# - Background regions with similar color to skin
# - Need balance based on application requirements

The key here is to ground your answers in evidence: run experiments, create visualizations, reference your code outputs, or explain the underlying mathematics.

You don't have to write essays—just conduct a small experiment, create a plot, or reference specific outputs from your code, and explain what you observe in 3-5 sentences. That's often enough.

But make sure that the words you use for analysis are your own and not LLM generated. This is a **strict requirement**. Try to understand and address the problems yourself in order to gain a better apreciation for the concepts you learn in class

---